In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install trimesh
!pip install fvcore iopath
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
pip install open3d

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch3d.io import load_obj
import torch.nn.functional as F
from pytorch3d.structures import Pointclouds
import os
import open3d as o3d
from pytorch3d.loss import chamfer_distance
from torch import nn
import torch
from torch import nn
import torch.nn.functional as F
from pytorch3d.loss import chamfer_distance
import numpy as np
import os
import open3d as o3d

Dataset processing


In [ ]:
class OBJDataset(Dataset):
    def __init__(self, csv_file, obj_dir, transform=None):
        # Load CSV with measurements
        self.measurements = pd.read_csv(csv_file)
        self.obj_dir = obj_dir
        self.transform = transform

        # Print the columns for debugging
        #print("Columns in CSV file:", self.measurements.columns.tolist())
        
    def __len__(self):
        return len(self.measurements)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Extract relevant fields from the row
        row = self.measurements.iloc[idx]
        processor = row['Processor']
        subject = row['Subject']
        measuring_station = row['Measuring station']
        # Construct the search prefix for the avatar file
        file_prefix = (f"{subject}_R1S1_{measuring_station}_{processor}" 
                       if "SizeStream" in processor and "SS20" in measuring_station else 
                       f"{subject}_R1_{measuring_station}_{processor}")

        # Search for the file starting with the file_prefix in the obj_dir
        avatar_file_name = None
        for file in os.listdir(self.obj_dir):
            if file.startswith(file_prefix) and file.endswith(".obj"):
                avatar_file_name = file
                break
        
        if avatar_file_name is None:
            raise FileNotFoundError(f"No .obj file found for index {idx} with prefix {file_prefix}")

        # Load the .obj file
        obj_path = os.path.join(self.obj_dir, avatar_file_name)
        verts, faces, _ = load_obj(obj_path)
        pointcloud = Pointclouds(points=[verts])

        measurement_cols = [
            'Neck girth', 'Back neck point to waist', 'Upper arm girth R',
            'Upper arm girth L', 'Back neck point to wrist R', 
            'Back neck point to wrist L', 'Across back shoulder width', 
            'Bust girth', 'Waist girth', 'Hip girth', 
            'Thigh girth R', 'Thigh girth L', 'Total crotch length', 
            'Inside leg height', 'gendre'
        ]

        # Check for missing columns
        missing_cols = [col for col in measurement_cols if col not in self.measurements.columns]
        if missing_cols:
            raise KeyError(f"The following columns are missing in the DataFrame: {missing_cols}")

        # Extract measurements
        measurements = row[measurement_cols]

        # Convert categorical 'gendre' to numerical (1 for Male, 0 for Female)
        #measurements['gendre'] = measurements['gendre'].replace({'Male': 1, 'Female': 0})
        measurements = measurements.replace({'Male': 1, 'Female': 0})

        # Ensure only numeric values are included and convert to float
        try:
            measurements = measurements.astype('float').values
        except ValueError as e:
            print(f"Error converting measurements at index {idx}: {e}")
            measurements = np.zeros(len(measurements))  # Use a zero vector or other handling

        # Package the pointcloud and measurements in a sample
        sample = {'pointcloud': pointcloud.points_packed(), 
                  'measurements': torch.tensor(measurements, dtype=torch.float)}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [ ]:
def pad_pointclouds(pointclouds_list):
    max_points = 10000  # Max number of points across the dataset
    padded_pointclouds = [F.pad(pc, (0, 0, 0, max_points - pc.shape[0]), "constant", 0) for pc in pointclouds_list]
    return torch.stack(padded_pointclouds)

# Custom collate function for DataLoader
def custom_collate(batch):
    pointclouds_list = [item['pointcloud'] for item in batch]
    measurements_list = [item['measurements'] for item in batch]
    padded_pointclouds = pad_pointclouds(pointclouds_list)
    measurements = torch.stack(measurements_list)
    return {'pointcloud': padded_pointclouds, 'measurements': measurements}


In [ ]:
csv_file_path = '/kaggle/input/ieee-dataset/data IEEE/Body Measurements_output_modified2.csv'
obj_dir_path = '/kaggle/input/ieee-dataset/data IEEE/3D_AVATARS'
obj_dataset = OBJDataset(csv_file=csv_file_path, obj_dir=obj_dir_path)
data_loader = DataLoader(obj_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate)


CGAN model creation:

Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, num_points, measurement_dim):
        super(Generator, self).__init__()
        self.num_points = num_points
        
        self.fc1 = nn.Sequential(
            nn.Linear(measurement_dim, 512),
            nn.GroupNorm(32, 512),  # 32 groups
            nn.SiLU(),
            nn.Linear(512, 1024),
            nn.GroupNorm(32, 1024),
            nn.SiLU(),
            nn.Linear(1024, 2048),
            nn.GroupNorm(32, 2048),
            nn.SiLU(),
            nn.Linear(2048, 4096),
            nn.GroupNorm(32, 4096),
            nn.SiLU()
        )


        
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_points * 3),
            nn.Tanh()  # Output between -1 and 1
        )

    def forward(self, measurements):
        x = self.fc1(measurements)
        pointcloud = self.fc2(x).view(-1, self.num_points, 3)
        return pointcloud

Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_points, measurement_dim):
        super(Discriminator, self).__init__()
        self.fc1_pointcloud = nn.Sequential(
            nn.Linear(num_points * 3, 4096),
            nn.BatchNorm1d(4096),
            nn.LeakyReLU(0.2),
            nn.Linear(4096, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2)
        )
        
        self.fc1_measurements = nn.Sequential(
            nn.Linear(measurement_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(1024 + 512, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1),
            nn.Sigmoid()  # Output a probability
        )
    
    def forward(self, pointcloud, measurements):
        pointcloud_flat = pointcloud.view(pointcloud.size(0), -1)
        pc_features = self.fc1_pointcloud(pointcloud_flat)
        meas_features = self.fc1_measurements(measurements)
        combined_features = torch.cat((pc_features, meas_features), dim=1)
        return self.fc2(combined_features)
   

Loss function for cGAN

In [ ]:
def gan_loss(discriminator, real_pc, fake_pc, real_measurements):
    # Adversarial loss
    real_preds = discriminator(real_pc, real_measurements)
    fake_preds = discriminator(fake_pc, real_measurements.detach())
    
    real_loss = F.binary_cross_entropy(real_preds, torch.ones_like(real_preds))
    fake_loss = F.binary_cross_entropy(fake_preds, torch.zeros_like(fake_preds))
    
    return real_loss + fake_loss

# Reconstruction + Adversarial Loss for Generator
def generator_loss(discriminator, fake_pc, real_pc, measurements):
    fake_preds = discriminator(fake_pc, measurements)
    adv_loss = F.binary_cross_entropy(fake_preds, torch.ones_like(fake_preds))
    
    # Chamfer loss for point cloud similarity
    chamfer_loss, _ = chamfer_distance(fake_pc, real_pc)
    
    return adv_loss + chamfer_loss


In [ ]:
def save_reconstructed_pointcloud_as_obj(generator, epoch, device, example_measurements, output_dir='/kaggle/working/'):
    generator.eval()
    with torch.no_grad():
        measurements = torch.tensor(example_measurements, dtype=torch.float32).to(device).unsqueeze(0)
        recon_pointcloud = generator(measurements)
        recon_pointcloud = recon_pointcloud[0].cpu().numpy()

        o3d_pc = o3d.geometry.PointCloud()
        o3d_pc.points = o3d.utility.Vector3dVector(recon_pointcloud)
        o3d_pc.estimate_normals()

        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            o3d_pc, depth=14
        )
        
        vertices_to_remove = densities < np.quantile(densities, 0.2)
        mesh.remove_vertices_by_mask(vertices_to_remove)

        obj_file_path = os.path.join(output_dir, f'reconstructed_mesh_cGAN_data1500_epoch_{epoch}.obj')
        o3d.io.write_triangle_mesh(obj_file_path, mesh)
        print(f'Saved .obj file at: {obj_file_path}')
        
        return obj_file_path

Training function for cGAN

In [ ]:
def train_gan(num_epochs, train_loader, generator, discriminator, g_optimizer, d_optimizer, device, example_measurements, checkpoint_path=None):
    if checkpoint_path:
        generator.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print(f"Loaded pretrained weights from {checkpoint_path}")

    generator.to(device)
    discriminator.to(device)

    for epoch in range(num_epochs):
        print('Epoch:', epoch)
        generator.train()
        discriminator.train()
        g_total_loss = 0
        d_total_loss = 0
        
        for data in train_loader:
            pointcloud = data['pointcloud'].to(device).float()
            measurements = data['measurements'].to(device).float()

            # Train Discriminator
            d_optimizer.zero_grad()
            fake_pc = generator(measurements)
            d_loss = gan_loss(discriminator, pointcloud, fake_pc, measurements)
            d_loss.backward()
            d_optimizer.step()
            d_total_loss += d_loss.item()
            # Train Generator
            g_optimizer.zero_grad()
            fake_pc = generator(measurements)
            g_loss = generator_loss(discriminator, fake_pc, pointcloud, measurements)
            g_loss.backward()
            g_optimizer.step()
            g_total_loss += g_loss.item()

        print(f'Epoch {epoch}, Generator Loss: {g_total_loss / len(train_loader)}, Discriminator Loss: {d_total_loss / len(train_loader)}')

        if epoch % 5 == 0:
            save_path = f'/kaggle/working/generator_cGAN_dataIEEE_epoch_{epoch}.pth'
            torch.save(generator.state_dict(), save_path)
            print(f'Model saved at {save_path}')
            save_reconstructed_pointcloud_as_obj(generator, epoch, device, example_measurements)


In [ ]:
# Initialize generator, discriminator, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(10000, 15)
discriminator = Discriminator(10000, 15)

g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-5)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-5)

example_measurements = [361.694,427.565,313.649,309.834,817.655,812.048,407.644,960.798,751.811,949.006,558.693,556.379,754.833,775.689,1]

train_gan(182, data_loader, generator, discriminator, g_optimizer, d_optimizer, device, example_measurements)
